In [1]:
import numpy as np
import pandas as pd
import os
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.externals import joblib
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import confusion_matrix

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.learning_curve import learning_curve
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

/Users/Sam/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/Sam/anaconda2/lib/python2.7/site-packages/sklearn/learning_curve.py:23: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the functions are moved. This module will be removed in 0.20
  DeprecationWarning)


In [2]:
os.chdir('/Users/Sam/ds/metis/metisgh/sf16_ds4/local/Project3')

In [3]:
df = pd.read_csv('default_of_credit_card_clients.csv')
df = df.dropna()

In [4]:
df = df.drop('ID', axis = 1)

In [5]:
df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,20000.0,2.0,2.0,1.0,24.0,2.0,2.0,-1.0,-1.0,-2.0,...,0.0,0.0,0.0,0.0,689.0,0.0,0.0,0.0,0.0,1.0
1,120000.0,2.0,2.0,2.0,26.0,-1.0,2.0,0.0,0.0,0.0,...,3272.0,3455.0,3261.0,0.0,1000.0,1000.0,1000.0,0.0,2000.0,1.0
2,90000.0,2.0,2.0,2.0,34.0,0.0,0.0,0.0,0.0,0.0,...,14331.0,14948.0,15549.0,1518.0,1500.0,1000.0,1000.0,1000.0,5000.0,0.0
3,50000.0,2.0,2.0,1.0,37.0,0.0,0.0,0.0,0.0,0.0,...,28314.0,28959.0,29547.0,2000.0,2019.0,1200.0,1100.0,1069.0,1000.0,0.0
4,50000.0,1.0,2.0,1.0,57.0,-1.0,0.0,-1.0,0.0,0.0,...,20940.0,19146.0,19131.0,2000.0,36681.0,10000.0,9000.0,689.0,679.0,0.0


In [6]:
df['default payment next month'] = df['default payment next month'].replace(to_replace=0, value="Paid")
df['default payment next month'] = df['default payment next month'].replace(to_replace=1, value="Default")
target_names = ["Default", "Paid"]

# Feature Scaling Limit Balance

In [7]:
df['LIMIT_BAL'] = df['LIMIT_BAL']/1000

In [8]:
#makes the percentage columns I was talking about
def percent_maker(df):
    for i in range(1,7):
        df[('pct_paid_{}'.format(i))] = df[('PAY_AMT{}'.format(i))] / df[('BILL_AMT{}'.format(i))]

percent_maker(df)

#replaces null and infinite values
df = df.replace({None:0, np.inf:1})

#new X features for modeling...
X = df[['LIMIT_BAL', 'SEX', 'EDUCATION', 'MARRIAGE', 'AGE','pct_paid_1', 'pct_paid_2', 'pct_paid_3',
       'pct_paid_4', 'pct_paid_5', 'pct_paid_6']]

target = df['default payment next month']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, target, test_size = 0.3, random_state = 42)

In [10]:
X.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,pct_paid_1,pct_paid_2,pct_paid_3,pct_paid_4,pct_paid_5,pct_paid_6
0,20.0,2.0,2.0,1.0,24.0,0.000000,0.222115,0.000000,0.000000,0.000000,0.000000
1,120.0,2.0,2.0,2.0,26.0,0.000000,0.579710,0.372856,0.305623,0.000000,0.613309
2,90.0,2.0,2.0,2.0,34.0,0.051917,0.106937,0.073752,0.069779,0.066899,0.321564
3,50.0,2.0,2.0,1.0,37.0,0.042562,0.041859,0.024345,0.038850,0.036914,0.033844
4,50.0,1.0,2.0,1.0,57.0,0.232099,6.469312,0.279057,0.429799,0.035987,0.035492


In [11]:
modellist = [KNeighborsClassifier(), LogisticRegression(), GaussianNB(), DecisionTreeClassifier(), RandomForestClassifier()]

In [12]:
for model in modellist:
    modelfit = model.fit(X_train, y_train)
    modelypred = modelfit.predict(X_test)
    print str(model).split("(")[0] + ": " + str(round(accuracy_score(y_test, modelypred),3))
    print confusion_matrix(y_test, model.predict(X_test))

KNeighborsClassifier: 0.738
[[ 279 1681]
 [ 674 6366]]
LogisticRegression: 0.782
[[   0 1960]
 [   0 7040]]
GaussianNB: 0.234
[[1931   29]
 [6865  175]]
DecisionTreeClassifier: 0.683
[[ 626 1334]
 [1516 5524]]
RandomForestClassifier: 0.755
[[ 533 1427]
 [ 782 6258]]


# Feature Selection

In [60]:
def allfeature(model):
    arr = []
    X_train, X_test, y_train, y_test = train_test_split(X, target, test_size = 0.3, random_state = 42)
    modelfit = model.fit(X_train, y_train)
    modely_pred = modelfit.predict(X_test)
    arr.append(round(accuracy_score(y_test, modely_pred),3))
    arr.append(round(recall_score(y_test, modely_pred, labels = ["Paid", "Default"], pos_label= "Default"),3))
    arr.append(round(precision_score(y_test, modely_pred, labels = ["Paid", "Default"], pos_label = "Default"),3))
    return arr

In [66]:
y_test.value_counts()

Paid       7040
Default    1960
Name: default payment next month, dtype: int64

In [65]:
for model in modellist:
    print model
    print allfeature(model)
    print confusion_matrix(y_test, model.predict(X_test))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
[0.753, 0.173, 0.361]
[[ 340 1620]
 [ 602 6438]]
LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
[0.782, 0.0, 0.0]
[[   0 1960]
 [   0 7040]]
GaussianNB(priors=None)
[0.378, 0.883, 0.244]
[[1730  230]
 [5370 1670]]
DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')
[0.727, 0.42, 0.384]
[[ 823 1137]
 [1318 5722]]
RandomForestClassifier(bootstrap=True, class

In [18]:
def onefeature(model, name):
    arr = []
    X = df[[name]]
    X_train, X_test, y_train, y_test = train_test_split(X, target, test_size = 0.3, random_state = 42)
    modelfit = model.fit(X_train, y_train)
    modely_pred = modelfit.predict(X_test)
    arr.append(round(accuracy_score(y_test, modely_pred),3))
    arr.append(round(recall_score(y_test, modely_pred, labels = ["Paid", "Default"], pos_label= "Default"),3))
    arr.append(round(precision_score(y_test, modely_pred, labels = ["Paid", "Default"], pos_label = "Default"),3))
    return arr

In [19]:
colnames = df.columns.tolist()
colnames.pop()

'default payment next month'

In [22]:
onemodel = modellist[1]

In [23]:
for title in colnames:
    print title
    print onefeature(onemodel, title)

LIMIT_BAL
[0.782, 0.0, 0.0]
SEX
[0.782, 0.0, 0.0]
EDUCATION
[0.782, 0.0, 0.0]
MARRIAGE
[0.782, 0.0, 0.0]
AGE
[0.782, 0.0, 0.0]
PAY_0
[0.82, 0.319, 0.687]
PAY_2
[0.785, 0.044, 0.577]
PAY_3
[0.784, 0.035, 0.575]
PAY_4
[0.785, 0.033, 0.621]
PAY_5
[0.786, 0.036, 0.648]
PAY_6
[0.783, 0.011, 0.667]
BILL_AMT1
[0.769, 0.014, 0.158]
BILL_AMT2
[0.765, 0.015, 0.14]
BILL_AMT3
[0.766, 0.02, 0.172]
BILL_AMT4
[0.767, 0.016, 0.153]
BILL_AMT5
[0.77, 0.02, 0.203]
BILL_AMT6
[0.768, 0.021, 0.197]
PAY_AMT1
[0.782, 0.0, 0.0]
PAY_AMT2
[0.782, 0.0, 0.0]
PAY_AMT3
[0.782, 0.0, 0.0]
PAY_AMT4
[0.782, 0.0, 0.0]
PAY_AMT5
[0.782, 0.0, 0.0]
PAY_AMT6
[0.782, 0.0, 0.0]
